In [1]:
ROOT_DIR = r" " # Root directory of the project

In [ ]:
import os
from ultralytics import YOLO
model = YOLO("yolov8n.pt")
results = model.train(data=os.path.join(ROOT_DIR, "config.yaml"), epochs=100)

In [ ]:
!yolo task=detect mode=predict model=" " conf=0.70 source=" " # Path to the image to be detected and the model to be used

In [ ]:
import cv2
import numpy as np
from ultralytics import YOLO

def toggle_fullscreen(window_name):
    current_state = cv2.getWindowProperty(window_name, cv2.WND_PROP_FULLSCREEN)
    cv2.setWindowProperty(window_name, cv2.WND_PROP_FULLSCREEN, 
                          cv2.WINDOW_NORMAL if current_state == cv2.WINDOW_FULLSCREEN else cv2.WINDOW_FULLSCREEN)

def draw_gui_bar(img, window_width):
    bar_height = 30
    cv2.rectangle(img, (0, 0), (window_width, bar_height), (60, 60, 60), -1)
    
    # Draw close button
    button_size = 20
    button_padding = 5
    button_pos = (window_width - button_size - button_padding, button_padding)
    cv2.rectangle(img, button_pos, (button_pos[0] + button_size, button_pos[1] + button_size), (200, 0, 0), -1)
    
    cv2.line(img, (button_pos[0] + 5, button_pos[1] + 5), 
             (button_pos[0] + button_size - 5, button_pos[1] + button_size - 5), (255, 255, 255), 2)
    cv2.line(img, (button_pos[0] + button_size - 5, button_pos[1] + 5), 
             (button_pos[0] + 5, button_pos[1] + button_size - 5), (255, 255, 255), 2)
    
    return bar_height, button_pos, button_size

def is_mouse_over_button(mouse_pos, button_pos, button_size):
    return (button_pos[0] < mouse_pos[0] < button_pos[0] + button_size and
            button_pos[1] < mouse_pos[1] < button_pos[1] + button_size)

model = YOLO(' ') # Path to the model to be used

video_path = r" " # Path to the video to be detected
cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    print("Error opening video stream or file")

window_name = 'Video Detection'
cv2.namedWindow(window_name, cv2.WINDOW_NORMAL)
cv2.setWindowProperty(window_name, cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)

mouse_x, mouse_y = 0, 0
is_mouse_down = False
should_exit = False

def mouse_callback(event, x, y, flags, param):
    global mouse_x, mouse_y, is_mouse_down, should_exit
    mouse_x, mouse_y = x, y
    
    if event == cv2.EVENT_LBUTTONDOWN:
        is_mouse_down = True
    elif event == cv2.EVENT_LBUTTONUP:
        if is_mouse_down and is_mouse_over_button((x, y), close_button_pos, close_button_size):
            should_exit = True
        is_mouse_down = False

cv2.setMouseCallback(window_name, mouse_callback)

screen_width = cv2.getWindowImageRect(window_name)[2]
screen_height = cv2.getWindowImageRect(window_name)[3]

while cap.isOpened() and not should_exit:
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.resize(frame, (screen_width, screen_height))

    results = model(frame)
    annotated_frame = results[0].plot()

    bar_height, close_button_pos, close_button_size = draw_gui_bar(annotated_frame, screen_width)

    if is_mouse_over_button((mouse_x, mouse_y), close_button_pos, close_button_size):
        annotated_frame = cv2.circle(annotated_frame, (mouse_x, mouse_y), 5, (0, 0, 255), -1)
    
    cv2.imshow(window_name, annotated_frame)

    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'):
        break
    elif key == ord('f'):
        toggle_fullscreen(window_name)

cap.release()
cv2.destroyAllWindows()